What variables should be used for training. We look at the performance of the default training vs the training with each variable removed. If the performance improves when we remove a variable, we know that variable is making the training worse. Further, by looking at the change in performance we can rank the variables to first order.

# Initalization

In [1]:
from bdt_training_scikit_tools import load_trimmed_sample, \
    test_train_samples, prep_samples, default_training, calc_performance, get_fraction_of_events, \
    default_training_variable_list
import matplotlib.pyplot as plt
plt.rc('font', size=14)
from matplotlib.colors import LogNorm
import pandas as pd
import numpy as np
import multiprocessing as mp
import itertools

# Load Data Samples

In [2]:
all_events_all = load_trimmed_sample("133")

Job 133:
  BIB: 800000 events
  Multijet: 800000 events
  Signal: 800000 events
  [800000, 800000, 473600]


In [3]:
all_events = get_fraction_of_events(all_events_all, 200000)
print ([len(e.index) for e in all_events])

[200387, 199675, 196736]


# Determine Full Variable List
We have to determine what variable list we want to start with before we start removing them according to their performance. Reasons for removal:

- Weight variables are MC only things
- Variables that have nothing to do with phyisics (like run #)
- DR to closest track because it is used later in the analysis
- JetPhi because physics *shouldn't* depend on that

In [4]:
all_events_all[0].columns

Index(['RunNumber', 'Weight', 'WeightMCEvent', 'WeightXSection',
       'WeightFlatten', 'mc_Lxy', 'mc_Lz', 'MHTOverHT', 'JetPt', 'JetPhi',
       'CalRatio', 'JetEta', 'NTracks', 'SumPtOfAllTracks', 'MaxTrackPt',
       'EventNumber', 'JetET', 'JetWidth', 'JetDRTo2GeVTrack', 'EnergyDensity',
       'HadronicLayer1Fraction', 'JetLat', 'JetLong', 'FirstClusterRadius',
       'NumberOfClusters', 'ShowerCenter', 'BIBDeltaTimingM',
       'BIBDeltaTimingP', 'FirstCellTiming', 'InteractionsPerCrossing',
       'RPredictedLxy', 'RPredictedLz', 'PredictedLxy', 'PredictedLz',
       'PredictedLxyHighEta', 'PredictedLxyLowEta', 'PredictedLzHighEta',
       'PredictedLzLowEta'],
      dtype='object')

In [5]:
variable_list = set(all_events_all[0].columns) - set(['RunNumber', 'Weight', 'WeightMCEvent', 'WeightXSection', 'WeightFlatten', 'mc_Lxy', 'mc_Lz', 'MHTOverHT', 'JetPhi', 'EventNumber', 'InteractionsPerCrossing', 'JetDRTo2GeVTrack', 'JetET'])
variable_list

{'BIBDeltaTimingM',
 'BIBDeltaTimingP',
 'CalRatio',
 'EnergyDensity',
 'FirstCellTiming',
 'FirstClusterRadius',
 'HadronicLayer1Fraction',
 'JetEta',
 'JetLat',
 'JetLong',
 'JetPt',
 'JetWidth',
 'MaxTrackPt',
 'NTracks',
 'NumberOfClusters',
 'PredictedLxy',
 'PredictedLxyHighEta',
 'PredictedLxyLowEta',
 'PredictedLz',
 'PredictedLzHighEta',
 'PredictedLzLowEta',
 'RPredictedLxy',
 'RPredictedLz',
 'ShowerCenter',
 'SumPtOfAllTracks'}

# Drop First Variable

In [6]:
%%writefile get_training_performance.py
from bdt_training_scikit_tools import load_default_samples, default_training_variable_list, \
    test_train_samples, prep_samples, default_training, calc_performance
    
def do_training (vlist):
    all_events, training_list = vlist
    return get_training_performance (all_events, training_list)
    
def get_training_performance (all_events, training_list):
    '''Run a training with the set of varaibles given. Return a performance table.'''
    
    # Split into testing and training samples
    train, test = test_train_samples(all_events)
        
    # Prep samples for training
    all_events, all_events_class, training_weight, evaluation_weight = prep_samples(train[0], train[1], train[2], training_variable_list=training_list)
    
    # Run training
    bdt = default_training(all_events, training_weight, all_events_class, estimators=400)
    
    # Create a thing of all the results
    return {tuple(training_list): calc_performance(bdt, test, training_variables = training_list)}

Overwriting get_training_performance.py


In [7]:
def all_but_one (vlist, number_to_drop = 1):
    '''Return vlist and vlist with each item removed
    
    Arguments:
        vlist - the source list or tuple
        number_to_drop - return a list that is len(vlist)-number_to_drop - all possible combinations
        
    Returns:
        A list of tuples that have len(vlist)-number_to_drop items.
    
    '''
    var_training_list = itertools.combinations(vlist, len(vlist)-number_to_drop)
    var_training_list = list(var_training_list) + [tuple(vlist)]
    return var_training_list

In [8]:
import get_training_performance
pool = mp.Pool(processes=10)

def unused_var(original_list, used_list):
    r = tuple(i for i in original_list if i not in used_list)
    r = r if len(r) != 0 else ('None',)
    return r

def calc_var_removal(all_events, training_list = default_training_variable_list):
    r_drop_first = pool.map(get_training_performance.do_training,
             [(all_events, tvar_list) for tvar_list in all_but_one(training_list)])

    one_dict = {}
    for kp in r_drop_first:
        one_dict.update(kp)

    return pd.DataFrame({unused_var(training_list, k):one_dict[k] for k in one_dict}).T

In [9]:
def remove_all_variables(all_events, training_list = variable_list):
    done = False
    count = 0
    rlist = []
    while not done:
        count = count + 1
        print ("Iteration #{0}".format(count))
        print ('  Training with variable list:')
        print ('  ' + str(training_list))
        result = calc_var_removal(all_events, training_list=training_list)
        result_sorted = result.sort_values("HSSSsqrtB")
        rlist = rlist + [result_sorted]
        print (result_sorted.HSSSsqrtB)
        last_var_name = result_sorted.index[-1][0]
        done = last_var_name == 'None'
        if not done:
            training_list = training_list - set([last_var_name])
    return rlist

In [10]:
%%time
#r_drop_first = calc_var_removal(all_events)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [11]:
#print (r_drop_first.sort_values("HSSSsqrtB").HSSSsqrtB)

In [12]:
%%time
r = remove_all_variables(all_events, variable_list)

Iteration #1
  Training with variable list:
  {'PredictedLz', 'HadronicLayer1Fraction', 'SumPtOfAllTracks', 'ShowerCenter', 'RPredictedLxy', 'JetEta', 'PredictedLxy', 'PredictedLxyHighEta', 'RPredictedLz', 'FirstClusterRadius', 'BIBDeltaTimingP', 'JetWidth', 'JetLat', 'NumberOfClusters', 'CalRatio', 'PredictedLzHighEta', 'FirstCellTiming', 'MaxTrackPt', 'NTracks', 'JetPt', 'EnergyDensity', 'JetLong', 'PredictedLzLowEta', 'BIBDeltaTimingM', 'PredictedLxyLowEta'}
JetPt                     147.028807
FirstCellTiming           147.450753
CalRatio                  162.559416
NTracks                   164.253331
RPredictedLxy             170.724881
PredictedLxy              171.902400
JetWidth                  173.856741
MaxTrackPt                203.645356
HadronicLayer1Fraction    210.197526
ShowerCenter              211.001034
PredictedLxyHighEta       211.523058
JetLat                    216.511655
PredictedLxyLowEta        217.477752
SumPtOfAllTracks          217.925181
BIBDeltaTimingP 

In [13]:
len(r)

3

In [14]:
r[-1]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,1.805890e+06,0.965245,48.070671,66925.0,66925.0,64599.0,2134.0,192.0,876915.341987,0.943811,...,148689.0,2148.0,2340.0,0.958252,1.268791e+06,66589.0,6.404988e+07,1.799186e+06,874781.341987,6.137591e+07
CalRatio,3.199952e+06,0.970908,36.324081,66925.0,66925.0,64978.0,1714.0,233.0,781812.204634,0.913984,...,143990.0,2631.0,2864.0,0.938031,1.122661e+06,66589.0,6.404988e+07,3.189032e+06,780098.204634,6.008075e+07
FirstCellTiming,2.788562e+06,0.969668,38.861640,66925.0,66925.0,64895.0,1823.0,207.0,751540.189834,0.902933,...,142249.0,2609.0,2816.0,0.944955,1.140547e+06,66589.0,6.404988e+07,2.775879e+06,749717.189834,6.052428e+07
NTracks,2.709389e+06,0.976660,39.709673,66925.0,66925.0,65363.0,1386.0,176.0,730791.278560,0.930501,...,146592.0,1944.0,2120.0,0.946451,1.316584e+06,66589.0,6.404988e+07,2.700384e+06,729405.278560,6.062009e+07
PredictedLxy,2.668618e+06,0.972596,39.845359,66925.0,66925.0,65091.0,1634.0,200.0,586365.778552,0.914594,...,144086.0,2635.0,2835.0,0.949375,1.142035e+06,66589.0,6.404988e+07,2.657798e+06,584731.778552,6.080735e+07
SumPtOfAllTracks,2.655034e+06,0.973104,39.968025,66925.0,66925.0,65125.0,1612.0,188.0,577477.017636,0.914746,...,144110.0,2584.0,2772.0,0.949726,1.155367e+06,66589.0,6.404988e+07,2.644187e+06,575865.017636,6.082983e+07
PredictedLxyLowEta,2.643388e+06,0.972551,40.033216,66925.0,66925.0,65088.0,1629.0,208.0,420199.676127,0.912486,...,143754.0,2578.0,2786.0,0.952369,1.155668e+06,66589.0,6.404988e+07,2.632179e+06,418570.676127,6.099913e+07
PredictedLxyHighEta,2.526305e+06,0.972850,40.962974,66925.0,66925.0,65108.0,1618.0,199.0,407366.855614,0.914219,...,144027.0,2635.0,2834.0,0.954392,1.148273e+06,66589.0,6.404988e+07,2.515426e+06,405748.855614,6.112870e+07
PredictedLzHighEta,2.334141e+06,0.972626,42.606000,66925.0,66925.0,65093.0,1631.0,201.0,400658.375908,0.914809,...,144120.0,2645.0,2846.0,0.957496,1.149576e+06,66589.0,6.404988e+07,2.323365e+06,399027.375908,6.132749e+07
JetEta,2.347869e+06,0.972835,42.490392,66925.0,66925.0,65107.0,1624.0,194.0,392310.559119,0.914809,...,144120.0,2593.0,2787.0,0.957412,1.161579e+06,66589.0,6.404988e+07,2.337041e+06,390686.559119,6.132215e+07


In [15]:
r[-2]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,1.941957e+06,0.968323,46.503829,66925.0,66925.0,64805.0,1919.0,201.0,1.066447e+06,0.939127,...,147951.0,2109.0,2310.0,0.953177,1.270240e+06,66589.0,6.404988e+07,1.934476e+06,1.064528e+06,6.105087e+07
FirstCellTiming,2.639046e+06,0.969757,39.951021,66925.0,66925.0,64901.0,1821.0,203.0,7.611635e+05,0.905745,...,142692.0,2642.0,2845.0,0.947132,1.137333e+06,66589.0,6.404988e+07,2.626839e+06,7.593425e+05,6.066370e+07
NTracks,2.329069e+06,0.977168,42.851561,66925.0,66925.0,65397.0,1350.0,178.0,7.339082e+05,0.932500,...,146907.0,1930.0,2108.0,0.952335,1.328535e+06,66589.0,6.404988e+07,2.320365e+06,7.325582e+05,6.099695e+07
CalRatio,3.229938e+06,0.970968,36.157299,66925.0,66925.0,64982.0,1709.0,234.0,7.081936e+05,0.916441,...,144377.0,2521.0,2755.0,0.938707,1.145481e+06,66589.0,6.404988e+07,3.219295e+06,7.064846e+05,6.012410e+07
None,2.354058e+06,0.972865,42.435809,66925.0,66925.0,65109.0,1624.0,192.0,6.700938e+05,0.917323,...,144516.0,2579.0,2771.0,0.952973,1.159526e+06,66589.0,6.404988e+07,2.343612e+06,6.684698e+05,6.103780e+07
RPredictedLxy,2.364800e+06,0.972432,42.320453,66925.0,66925.0,65080.0,1652.0,193.0,6.503634e+05,0.917958,...,144616.0,2561.0,2754.0,0.953112,1.163270e+06,66589.0,6.404988e+07,2.354436e+06,6.487114e+05,6.104673e+07
SumPtOfAllTracks,2.331982e+06,0.973328,42.656495,66925.0,66925.0,65140.0,1587.0,198.0,6.207021e+05,0.916974,...,144461.0,2567.0,2765.0,0.954089,1.162143e+06,66589.0,6.404988e+07,2.321469e+06,6.191151e+05,6.110929e+07
PredictedLxy,2.568888e+06,0.972761,40.618300,66925.0,66925.0,65102.0,1624.0,199.0,5.819549e+05,0.917298,...,144512.0,2576.0,2775.0,0.950995,1.156285e+06,66589.0,6.404988e+07,2.558435e+06,5.803309e+05,6.091111e+07
ShowerCenter,2.442219e+06,0.972955,41.666663,66925.0,66925.0,65115.0,1621.0,189.0,4.354894e+05,0.915755,...,144269.0,2662.0,2851.0,0.955262,1.145888e+06,66589.0,6.404988e+07,2.431609e+06,4.338684e+05,6.118440e+07
BIBDeltaTimingP,2.358877e+06,0.972626,42.382016,66925.0,66925.0,65093.0,1642.0,190.0,4.355436e+05,0.917755,...,144584.0,2564.0,2754.0,0.956559,1.167476e+06,66589.0,6.404988e+07,2.348484e+06,4.339016e+05,6.126749e+07


# Look at high $\eta$ and low $\eta$ list

In [16]:
%%time
r_low_high = remove_all_variables(all_events, {'PredictedLxyLowEta', \
                                              'CalRatio', 'HadronicLayer1Fraction', 'EnergyDensity', \
                                              'JetEta', 'SumPtOfAllTracks', 'ShowerCenter', \
                                              'JetPt', 'NTracks', 'NumberOfClusters', \
                                              'FirstCellTiming', 'BIBDeltaTimingM', 'JetLat', 'JetWidth', 'FirstClusterRadius', \
                                              'PredictedLzLowEta', 'BIBDeltaTimingP', 'PredictedLzHighEta', \
                                              'PredictedLxyHighEta', 'JetLong', 'MaxTrackPt'})

Iteration #1
  Training with variable list:
  {'HadronicLayer1Fraction', 'SumPtOfAllTracks', 'ShowerCenter', 'JetEta', 'PredictedLxyHighEta', 'FirstClusterRadius', 'BIBDeltaTimingP', 'JetWidth', 'JetLat', 'NumberOfClusters', 'CalRatio', 'PredictedLzHighEta', 'FirstCellTiming', 'MaxTrackPt', 'NTracks', 'JetPt', 'EnergyDensity', 'JetLong', 'PredictedLzLowEta', 'BIBDeltaTimingM', 'PredictedLxyLowEta'}
JetPt                     128.442961
HadronicLayer1Fraction    157.527794
ShowerCenter              160.457798
JetLong                   161.555923
NumberOfClusters          162.096123
MaxTrackPt                162.402702
NTracks                   163.100311
SumPtOfAllTracks          163.280128
None                      163.511721
BIBDeltaTimingM           163.750333
PredictedLzLowEta         168.284459
JetWidth                  168.699996
BIBDeltaTimingP           169.149988
JetLat                    169.329569
FirstCellTiming           169.605331
EnergyDensity             171.226467
JetEta

JetPt                     111.317308
FirstCellTiming           177.809027
NTracks                   201.154912
None                      201.854129
HadronicLayer1Fraction    204.415598
JetWidth                  212.820248
PredictedLzHighEta        229.005201
FirstClusterRadius        236.903253
JetEta                    238.198207
NumberOfClusters          244.650115
PredictedLxyLowEta        252.648375
MaxTrackPt                271.688000
PredictedLxyHighEta       275.798230
SumPtOfAllTracks          277.863357
Name: HSSSsqrtB, dtype: float64
Iteration #10
  Training with variable list:
  {'NumberOfClusters', 'HadronicLayer1Fraction', 'PredictedLzHighEta', 'FirstCellTiming', 'PredictedLxyHighEta', 'MaxTrackPt', 'NTracks', 'FirstClusterRadius', 'JetPt', 'JetEta', 'JetWidth', 'PredictedLxyLowEta'}
JetPt                     105.913953
FirstCellTiming           125.741998
JetWidth                  175.804113
NTracks                   189.087155
HadronicLayer1Fraction    210.676505
Predict

In [17]:
r_low_high[-1]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,2.731542e+06,0.956728,38.741172,66925.0,66925.0,64029.0,2658.0,238.0,1.995538e+06,0.933839,...,147118.0,2039.0,2277.0,0.926369,1.243429e+06,66589.0,6.404988e+07,2.723158e+06,1.992880e+06,5.933384e+07
FirstCellTiming,5.659758e+06,0.940949,26.470092,66925.0,66925.0,62973.0,3706.0,246.0,2.732798e+05,0.536216,...,84476.0,2327.0,2573.0,0.908531,1.147197e+06,66589.0,6.404988e+07,5.589020e+06,2.695738e+05,5.819128e+07
JetWidth,4.294577e+06,0.963302,31.109337,66925.0,66925.0,64469.0,2174.0,282.0,5.886131e+05,0.881098,...,138809.0,2322.0,2604.0,0.924050,1.159827e+06,66589.0,6.404988e+07,4.278167e+06,5.864391e+05,5.918527e+07
MaxTrackPt,4.127858e+06,0.963571,31.740209,66925.0,66925.0,64487.0,2161.0,277.0,4.116332e+05,0.882107,...,138968.0,2710.0,2987.0,0.929407,1.089198e+06,66589.0,6.404988e+07,4.111995e+06,4.094722e+05,5.952841e+07
PredictedLzHighEta,4.505452e+06,0.964079,30.397084,66925.0,66925.0,64521.0,2114.0,290.0,4.173267e+05,0.889330,...,140106.0,2274.0,2564.0,0.923411,1.168031e+06,66589.0,6.404988e+07,4.490291e+06,4.152127e+05,5.914437e+07
NTracks,4.103194e+06,0.967516,31.965790,66925.0,66925.0,64751.0,1897.0,277.0,3.609882e+05,0.902870,...,142239.0,2145.0,2422.0,0.930537,1.211057e+06,66589.0,6.404988e+07,4.090037e+06,3.590912e+05,5.960075e+07
HadronicLayer1Fraction,4.457404e+06,0.961868,30.490375,66925.0,66925.0,64373.0,2287.0,265.0,3.190732e+05,0.886899,...,139723.0,2226.0,2491.0,0.925705,1.187966e+06,66589.0,6.404988e+07,4.441812e+06,3.167862e+05,5.929128e+07
PredictedLxyHighEta,4.732687e+06,0.963930,29.653768,66925.0,66925.0,64511.0,2127.0,287.0,2.894348e+05,0.890809,...,140339.0,2316.0,2603.0,0.921856,1.157296e+06,66589.0,6.404988e+07,4.717801e+06,2.873078e+05,5.904477e+07
NumberOfClusters,4.722503e+06,0.964557,29.705050,66925.0,66925.0,64553.0,2099.0,273.0,2.878441e+05,0.892403,...,140590.0,2251.0,2524.0,0.922037,1.175498e+06,66589.0,6.404988e+07,4.707803e+06,2.857451e+05,5.905633e+07
JetEta,5.079107e+06,0.963437,28.610001,66925.0,66925.0,64478.0,2150.0,297.0,2.721447e+05,0.889476,...,140129.0,2314.0,2611.0,0.916721,1.149085e+06,66589.0,6.404988e+07,5.064009e+06,2.699947e+05,5.871587e+07


# Look at full $\eta$ range predictions

In [18]:
r_full = remove_all_variables(all_events, {'PredictedLxy', \
                                              'CalRatio', 'HadronicLayer1Fraction', 'EnergyDensity', \
                                              'JetEta', 'SumPtOfAllTracks', 'ShowerCenter', \
                                              'JetPt', 'NTracks', 'NumberOfClusters', \
                                              'FirstCellTiming', 'BIBDeltaTimingM', 'JetLat', 'JetWidth', 'FirstClusterRadius', \
                                              'PredictedLz', 'BIBDeltaTimingP', \
                                              'JetLong', 'MaxTrackPt'})

Iteration #1
  Training with variable list:
  {'NumberOfClusters', 'PredictedLxy', 'CalRatio', 'HadronicLayer1Fraction', 'PredictedLz', 'FirstCellTiming', 'SumPtOfAllTracks', 'ShowerCenter', 'NTracks', 'FirstClusterRadius', 'MaxTrackPt', 'JetPt', 'BIBDeltaTimingP', 'EnergyDensity', 'JetEta', 'BIBDeltaTimingM', 'JetWidth', 'JetLat', 'JetLong'}
NTracks                   134.396543
JetPt                     137.859663
HadronicLayer1Fraction    159.864160
JetEta                    162.473386
JetWidth                  165.475649
SumPtOfAllTracks          167.090512
MaxTrackPt                167.475195
ShowerCenter              168.139958
PredictedLz               168.667466
None                      169.385510
PredictedLxy              170.623653
EnergyDensity             170.868948
NumberOfClusters          171.430412
BIBDeltaTimingP           174.156600
BIBDeltaTimingM           174.196528
FirstClusterRadius        174.371387
JetLong                   174.707323
JetLat                    

In [19]:
r_full[-1]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,2.826853e+06,0.956444,38.071170,66925.0,66925.0,64010.0,2669.0,246.0,1.652804e+06,0.916606,...,144403.0,2508.0,2754.0,0.930268,1.135388e+06,66589.0,6.404988e+07,2.816223e+06,1.650135e+06,5.958352e+07
NTracks,4.455811e+06,0.968681,30.711847,66925.0,66925.0,64829.0,1884.0,212.0,6.346272e+05,0.910887,...,143502.0,2173.0,2385.0,0.920739,1.207565e+06,66589.0,6.404988e+07,4.443945e+06,6.327432e+05,5.897319e+07
BIBDeltaTimingM,4.271320e+06,0.960538,31.104402,66925.0,66925.0,64284.0,2391.0,250.0,3.892307e+05,0.824052,...,129822.0,2417.0,2667.0,0.927668,1.150534e+06,66589.0,6.404988e+07,4.246018e+06,3.868397e+05,5.941702e+07
HadronicLayer1Fraction,4.601429e+06,0.959417,29.932953,66925.0,66925.0,64209.0,2461.0,255.0,3.923706e+05,0.879301,...,138526.0,2681.0,2936.0,0.922326,1.090247e+06,66589.0,6.404988e+07,4.585095e+06,3.899096e+05,5.907487e+07
BIBDeltaTimingP,4.548319e+06,0.962809,30.213643,66925.0,66925.0,64436.0,2248.0,241.0,2.875958e+05,0.808932,...,127440.0,2455.0,2696.0,0.924964,1.140994e+06,66589.0,6.404988e+07,4.520673e+06,2.853478e+05,5.924386e+07
JetEta,4.654511e+06,0.962899,29.869777,66925.0,66925.0,64442.0,2224.0,259.0,2.946137e+05,0.876781,...,138129.0,2627.0,2886.0,0.923027,1.100486e+06,66589.0,6.404988e+07,4.637726e+06,2.923897e+05,5.911976e+07
PredictedLz,4.700285e+06,0.961808,29.690307,66925.0,66925.0,64369.0,2313.0,243.0,2.846157e+05,0.880971,...,138789.0,2522.0,2765.0,0.922461,1.123618e+06,66589.0,6.404988e+07,4.684055e+06,2.823027e+05,5.908352e+07
PredictedLxy,4.254167e+06,0.960164,31.154924,66925.0,66925.0,64259.0,2424.0,242.0,2.755873e+05,0.878698,...,138431.0,2474.0,2716.0,0.929575,1.142452e+06,66589.0,6.404988e+07,4.237531e+06,2.731633e+05,5.953918e+07
NumberOfClusters,4.472993e+06,0.962734,30.464621,66925.0,66925.0,64431.0,2243.0,251.0,2.739018e+05,0.882856,...,139086.0,2758.0,3009.0,0.926168,1.081426e+06,66589.0,6.404988e+07,4.457296e+06,2.716588e+05,5.932092e+07
SumPtOfAllTracks,4.631761e+06,0.963317,29.956052,66925.0,66925.0,64470.0,2207.0,248.0,2.729207e+05,0.881828,...,138924.0,2654.0,2902.0,0.923708,1.098257e+06,66589.0,6.404988e+07,4.615798e+06,2.707137e+05,5.916337e+07


In [20]:
r_full[-2]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,3.111732e+06,0.962630,36.521331,66925.0,66925.0,64424.0,2258.0,243.0,1.458628e+06,0.932214,...,146862.0,2215.0,2458.0,0.928811,1.199927e+06,66589.0,6.404988e+07,3.103268e+06,1.456370e+06,5.949024e+07
PredictedLxy,4.128359e+06,0.967845,31.879043,66925.0,66925.0,64773.0,1919.0,233.0,5.648097e+05,0.899759,...,141749.0,2549.0,2782.0,0.926963,1.125647e+06,66589.0,6.404988e+07,4.115116e+06,5.628907e+05,5.937187e+07
BIBDeltaTimingM,4.446413e+06,0.968278,30.731481,66925.0,66925.0,64802.0,1872.0,251.0,5.160709e+05,0.906005,...,142733.0,2505.0,2756.0,0.922743,1.125795e+06,66589.0,6.404988e+07,4.434110e+06,5.141989e+05,5.910157e+07
None,4.248337e+06,0.968427,31.444590,66925.0,66925.0,64812.0,1864.0,249.0,5.082136e+05,0.906024,...,142736.0,2484.0,2733.0,0.925958,1.134462e+06,66589.0,6.404988e+07,4.236016e+06,5.063496e+05,5.930751e+07
NTracks,4.900633e+06,0.971999,29.385148,66925.0,66925.0,65051.0,1664.0,210.0,5.281245e+05,0.924521,...,145650.0,2037.0,2247.0,0.915422,1.236909e+06,66589.0,6.404988e+07,4.890779e+06,5.264605e+05,5.863264e+07
HadronicLayer1Fraction,4.477125e+06,0.966918,30.582887,66925.0,66925.0,64711.0,1970.0,244.0,4.113394e+05,0.905402,...,142638.0,2547.0,2791.0,0.923901,1.120118e+06,66589.0,6.404988e+07,4.464769e+06,4.093694e+05,5.917574e+07
JetEta,4.474856e+06,0.968487,30.640277,66925.0,66925.0,64816.0,1849.0,260.0,4.108624e+05,0.905631,...,142674.0,2514.0,2774.0,0.923942,1.123595e+06,66589.0,6.404988e+07,4.462503e+06,4.090134e+05,5.917836e+07
MaxTrackPt,4.156668e+06,0.968427,31.789431,66925.0,66925.0,64812.0,1872.0,241.0,4.111966e+05,0.907986,...,143045.0,2463.0,2704.0,0.928900,1.144152e+06,66589.0,6.404988e+07,4.144635e+06,4.093246e+05,5.949592e+07
JetWidth,4.245963e+06,0.967396,31.419895,66925.0,66925.0,64743.0,1929.0,253.0,3.759586e+05,0.904780,...,142540.0,2481.0,2734.0,0.928064,1.136834e+06,66589.0,6.404988e+07,4.233443e+06,3.740296e+05,5.944240e+07
SumPtOfAllTracks,4.039946e+06,0.968502,32.247880,66925.0,66925.0,64817.0,1863.0,245.0,3.763105e+05,0.906291,...,142778.0,2528.0,2773.0,0.931270,1.132710e+06,66589.0,6.404988e+07,4.027711e+06,3.744475e+05,5.964772e+07


# Using original Rachel $L_{xy}$ and $L_{z}$ predictions

In [21]:
r_rachel = remove_all_variables(all_events, {'RPredictedLxy', \
                                              'CalRatio', 'HadronicLayer1Fraction', 'EnergyDensity', \
                                              'JetEta', 'SumPtOfAllTracks', 'ShowerCenter', \
                                              'JetPt', 'NTracks', 'NumberOfClusters', \
                                              'FirstCellTiming', 'BIBDeltaTimingM', 'JetLat', 'JetWidth', 'FirstClusterRadius', \
                                              'RPredictedLz', 'BIBDeltaTimingP', \
                                              'JetLong', 'MaxTrackPt'})

Iteration #1
  Training with variable list:
  {'NumberOfClusters', 'CalRatio', 'HadronicLayer1Fraction', 'FirstCellTiming', 'SumPtOfAllTracks', 'RPredictedLz', 'ShowerCenter', 'RPredictedLxy', 'NTracks', 'FirstClusterRadius', 'JetPt', 'BIBDeltaTimingP', 'EnergyDensity', 'JetEta', 'BIBDeltaTimingM', 'MaxTrackPt', 'JetWidth', 'JetLat', 'JetLong'}
FirstCellTiming           145.551161
NTracks                   155.218947
JetPt                     158.215490
HadronicLayer1Fraction    162.964118
CalRatio                  166.952844
JetWidth                  168.263313
BIBDeltaTimingM           168.653280
JetEta                    168.772058
BIBDeltaTimingP           170.593249
ShowerCenter              170.846248
NumberOfClusters          170.897665
JetLat                    170.994720
EnergyDensity             171.067778
MaxTrackPt                171.782128
SumPtOfAllTracks          172.145175
None                      174.691754
FirstClusterRadius        175.566942
JetLong                 

In [22]:
len(r_rachel)

8

In [23]:
r_rachel[-1]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,2.355920e+06,0.959163,41.821600,66925.0,66925.0,64192.0,2539.0,194.0,1.254747e+06,0.912455,...,143749.0,2524.0,2718.0,0.943843,1.159560e+06,66589.0,6.404988e+07,2.344652e+06,1.252208e+06,6.045302e+07
NTracks,3.342243e+06,0.970265,35.518929,66925.0,66925.0,64935.0,1794.0,196.0,8.013635e+05,0.910633,...,143462.0,2289.0,2485.0,0.935519,1.202009e+06,66589.0,6.404988e+07,3.330453e+06,7.995695e+05,5.991986e+07
HadronicLayer1Fraction,3.619443e+06,0.962854,33.870990,66925.0,66925.0,64439.0,2268.0,218.0,7.232859e+05,0.880780,...,138759.0,2896.0,3114.0,0.932481,1.070284e+06,66589.0,6.404988e+07,3.603557e+06,7.210179e+05,5.972530e+07
FirstClusterRadius,3.349609e+06,0.965917,35.320854,66925.0,66925.0,64644.0,2071.0,210.0,7.112600e+05,0.884386,...,139327.0,2918.0,3128.0,0.936869,1.072912e+06,66589.0,6.404988e+07,3.334313e+06,7.091890e+05,6.000638e+07
RPredictedLz,3.407045e+06,0.965723,35.014822,66925.0,66925.0,64631.0,2075.0,219.0,6.805268e+05,0.877308,...,138212.0,3011.0,3230.0,0.936469,1.055383e+06,66589.0,6.404988e+07,3.390727e+06,6.784518e+05,5.998070e+07
EnergyDensity,3.399738e+06,0.965693,35.051348,66925.0,66925.0,64629.0,2074.0,222.0,6.257705e+05,0.883186,...,139138.0,2965.0,3187.0,0.937424,1.063563e+06,66589.0,6.404988e+07,3.384300e+06,6.236965e+05,6.004188e+07
CalRatio,3.835678e+06,0.963571,32.926918,66925.0,66925.0,64487.0,2185.0,253.0,5.682433e+05,0.877264,...,138205.0,3101.0,3354.0,0.931530,1.030229e+06,66589.0,6.404988e+07,3.819443e+06,5.660583e+05,5.966438e+07
BIBDeltaTimingP,3.198466e+06,0.969623,36.284431,66925.0,66925.0,64892.0,1811.0,222.0,3.825466e+05,0.746352,...,117581.0,3257.0,3479.0,0.944692,1.025844e+06,66589.0,6.404988e+07,3.161763e+06,3.807356e+05,6.050738e+07
BIBDeltaTimingM,3.733869e+06,0.967680,33.515116,66925.0,66925.0,64762.0,1942.0,221.0,3.650815e+05,0.757885,...,119398.0,3171.0,3392.0,0.936580,1.029995e+06,66589.0,6.404988e+07,3.698897e+06,3.631395e+05,5.998784e+07
SumPtOfAllTracks,3.230641e+06,0.965738,35.958641,66925.0,66925.0,64632.0,2068.0,225.0,4.739063e+05,0.877657,...,138267.0,3413.0,3638.0,0.942441,1.000786e+06,66589.0,6.404988e+07,3.214780e+06,4.718383e+05,6.036326e+07


In [24]:
r_rachel[-2]

,BIBBack,BIBEff,BIBSsqrtB,BIBTotalCount,BIBTotalWeight,BIBinBIB,BIBinHSS,BIBinMJ,HSSBack,HSSEff,...,HSSinHSS,HSSinMJ,MJBack,MJEff,MJSsqrtB,MJTotalCount,MJTotalWeight,MJinBIB,MJinHSS,MJinMJ
JetPt,2.392912e+06,0.959731,41.521644,66925.0,66925.0,64230.0,2505.0,190.0,1.277773e+06,0.909903,...,143347.0,2491.0,2681.0,0.942912,1.166383e+06,66589.0,6.404988e+07,2.381209e+06,1.275268e+06,6.039340e+07
HadronicLayer1Fraction,3.209670e+06,0.962346,35.949216,66925.0,66925.0,64405.0,2300.0,220.0,7.179748e+05,0.882183,...,138980.0,2809.0,3029.0,0.938960,1.092737e+06,66589.0,6.404988e+07,3.193918e+06,7.156748e+05,6.014028e+07
NTracks,3.924036e+06,0.970728,32.795905,66925.0,66925.0,64966.0,1766.0,193.0,7.440241e+05,0.911407,...,143584.0,2181.0,2374.0,0.927330,1.219024e+06,66589.0,6.404988e+07,3.912260e+06,7.422581e+05,5.939536e+07
RPredictedLz,3.566864e+06,0.965992,34.230916,66925.0,66925.0,64649.0,2070.0,206.0,6.882846e+05,0.878387,...,138382.0,2952.0,3158.0,0.933850,1.064362e+06,66589.0,6.404988e+07,3.550657e+06,6.862146e+05,5.981301e+07
EnergyDensity,3.083331e+06,0.965573,36.801340,66925.0,66925.0,64621.0,2079.0,225.0,6.229678e+05,0.882158,...,138976.0,2923.0,3148.0,0.942411,1.075824e+06,66589.0,6.404988e+07,3.067689e+06,6.208888e+05,6.036130e+07
FirstClusterRadius,3.712825e+06,0.965812,33.545087,66925.0,66925.0,64637.0,2081.0,207.0,6.099941e+05,0.884125,...,139286.0,2850.0,3057.0,0.932782,1.080564e+06,66589.0,6.404988e+07,3.697420e+06,6.079131e+05,5.974454e+07
BIBDeltaTimingP,3.266131e+06,0.970026,35.921549,66925.0,66925.0,64919.0,1783.0,223.0,4.034395e+05,0.752065,...,118481.0,3094.0,3317.0,0.943297,1.049045e+06,66589.0,6.404988e+07,3.230165e+06,4.016565e+05,6.041806e+07
CalRatio,4.028789e+06,0.963840,32.137060,66925.0,66925.0,64505.0,2168.0,252.0,5.505556e+05,0.880914,...,138780.0,2991.0,3243.0,0.928784,1.044622e+06,66589.0,6.404988e+07,4.013019e+06,5.483876e+05,5.948847e+07
BIBDeltaTimingM,3.380756e+06,0.967949,35.231733,66925.0,66925.0,64780.0,1925.0,220.0,3.783432e+05,0.759015,...,119576.0,3134.0,3354.0,0.941884,1.041679e+06,66589.0,6.404988e+07,3.345925e+06,3.764182e+05,6.032753e+07
SumPtOfAllTracks,3.525958e+06,0.965588,34.414531,66925.0,66925.0,64622.0,2086.0,217.0,4.676125e+05,0.878184,...,138350.0,3307.0,3524.0,0.937930,1.011977e+06,66589.0,6.404988e+07,3.510074e+06,4.655265e+05,6.007428e+07
